In [1]:
import time as t
import datetime as dt
import pandas as pd
import os
import sqlite3
from dotenv import load_dotenv
from telethon import TelegramClient, events

In [2]:
load_dotenv('../.env')
telegram_api_id = os.getenv('telegram_api_id')
telegram_api_hash = os.getenv('telegram_api_hash')
telegram_phone_number = os.getenv('telegram_phone_number')
pass2fa = os.getenv('telegram_pass2fa')

In [3]:
client = TelegramClient(session=None,
                        api_id=telegram_api_id,
                        auto_reconnect=True,
                        api_hash=telegram_api_hash)

In [4]:
await client.start(phone=telegram_phone_number, password=pass2fa)

Signed in successfully as Jack Ma; remember to not break the ToS or you will risk an account ban!


In [5]:
async for dialog in client.iter_dialogs():
    print(dialog.name, 'has ID', dialog.id)

Solana Scanner has ID -1002023951506
Telegram has ID 777000
Gambles 🎲 MadApes has ID -1001758611100
Solana New Liquidity Pools has ID -1002039712427
Degen Seals has ID -1001523523939
Marat Gudkov has ID 178008364
Jack Ma has ID 1223950100
Yield Board has ID -1001628892345


In [6]:
lp_chat = -1002039712427

In [7]:

## read last message
message =  await client.get_messages(lp_chat, limit=5)
message = pd.DataFrame({'messages':message})
message['id'] = message['messages'].apply(lambda x:x.id)
message['date'] = message['messages'].apply(lambda x:x.date)
message['text'] = message['messages'].apply(lambda x:x.text)
message['date'] = pd.to_datetime(message['date'])
message['date'] = message['date'].dt.tz_localize(None)

pattern = r'`([1-9A-HJ-NP-Za-km-z]{32,44})`'
message['address'] = message['text'].str.extract(pattern)
message['expected_launch_time'] = message['text'].str.extract(r'(?<=Launch:\*\* )(.*)')[0].str.strip('`')
message['mcap'] = message['text'].str.extract(r'(?<=Mcap:\*\* )(.*)')[0].str.strip('`')
message['liq'] = message['text'].str.extract(r'(?<=Liq:\*\* )(.*)')[0].str.strip('`')
message['s_mm'] = message['text'].str.extract(r'(?<=Mutable Metadata: )(.*)')
message['s_ma'] = message['text'].str.extract(r'(?<=Mint Authority: )(.*)')
message['s_fa'] = message['text'].str.extract(r'(?<=Freeze Authority: )(.*)')
message['s_s'] = message['text'].str.extract(r'(?<=Score: )(.*)')

In [8]:
## Parse
message.loc[message['s_mm'].str.contains('Yes'), 's_mm2'] = True
message.loc[message['s_mm'].str.contains('No'), 's_mm2'] = False
message.loc[message['s_ma'].str.contains('Yes'), 's_ma2'] = True
message.loc[message['s_ma'].str.contains('No'), 's_ma2'] = False
message.loc[message['s_fa'].str.contains('Yes'), 's_fa2'] = True
message.loc[message['s_fa'].str.contains('No'), 's_fa2'] = False
message.loc[message['s_s'].str.contains('Bad'), 's_q'] = 0
message.loc[message['s_s'].str.contains('Neutral'), 's_q'] = 1
message.loc[message['s_s'].str.contains('Good'), 's_q'] = 2
message['s_sni'] = message['s_s'].str.extract(r'([0-9]+)')
message['s_sni'] = message['s_sni'].fillna(0)

mult = {'K':1e3,'M':1e6,'B':1e9,'T':1e12}
message['mcap'] = message['mcap'].str.strip('[$\*]')
message['mcap_num'] = message['mcap'].str.extract('([0-9\.]+)').astype(float) * message['mcap'].str.extract('([A-Z])').replace(mult).fillna(1)
message['liq'] = message['liq'].str.strip(r" \[.*\]")
message['liq_num'] = message['liq'].str.extract('([0-9\.]+)').astype(float) * message['liq'].str.extract('([A-Z])').replace(mult).fillna(1)
message[['s_q','mcap_num','liq_num']] = message[['s_q','mcap_num','liq_num']].astype(int)

In [10]:
## get start time
tp = message['expected_launch_time'].str.extractall('([0-9]+)').unstack()
tp.columns = tp.columns.droplevel(0)
tp['hours'] = message['expected_launch_time'].str.contains('hour')
tp['sec'] = message['expected_launch_time'].str.contains('second')
tp.loc[tp['hours']==True, 'new'] = tp.loc[tp['hours']==True].apply(lambda x: dt.timedelta(hours=int(x[0]), minutes=int(x[1])), axis=1)
tp.loc[tp['sec']==True, 'new'] = tp.loc[tp['sec']==True].apply(lambda x: dt.timedelta(seconds=int(x[0])), axis=1)
tp.loc[tp['new'].isna(), 'new'] = tp.loc[tp['new'].isna()].apply(lambda x: dt.timedelta(minutes=int(x[0])), axis=1)
tp.loc[tp['new']>=dt.timedelta(days=30),'new'] = dt.timedelta(seconds=0)

message = pd.concat([message, tp[['new']]],axis=1)
message.loc[(message['expected_launch_time'].str.contains('ago')), 'expected_launch_time_ts'] = message['date'] - pd.to_timedelta(message['new'])
message.loc[(message['expected_launch_time'].str.contains('In')), 'expected_launch_time_ts'] = message['date'] + pd.to_timedelta(message['new'])


In [11]:
message = message.drop(columns=['messages','s_mm','s_ma','s_fa','s_s','mcap','liq','text','new'])

In [12]:
message

,id,date,address,expected_launch_time,s_mm2,s_ma2,s_fa2,s_q,s_sni,mcap_num,liq_num,expected_launch_time_ts
0,215605,2024-04-18 14:23:50,FybqbAftE88wSdr495baGGL1ajRSdP4218PYtm4GH6F4,In 2 minutes,False,False,False,2,0,2710,5430,2024-04-18 14:25:50
1,215604,2024-04-18 14:22:53,HJoaBRWFu44tamHWGX9csxiE9Z322WUMaEwVrUdckALH,6 seconds ago,True,True,False,0,3,678,1360,2024-04-18 14:22:47
2,215603,2024-04-18 14:22:29,7mjjbNaQBCZGLSNwbfWZH4tCwNw5VZcNqbQGj299qGk4,2 minutes ago,False,False,False,0,4,1460,0,2024-04-18 14:20:29
3,215602,2024-04-18 14:22:20,3TDqQCyxdc7pTNQZ2E2Vycdh1X8Wqrc7P3x2wMwoh4pz,In 1 minute,True,False,False,1,1,8820,17640,2024-04-18 14:23:20
4,215601,2024-04-18 14:22:16,FZ2FNQKY84oHx2y3vGe9cq9YymFSTzFFjqBatyN6bDpP,42 seconds ago,False,False,False,0,3,329,461,2024-04-18 14:21:34


In [13]:
conn = sqlite3.connect('../dbs/calls.db')
message.to_sql('calls', conn, if_exists='append', index=False)
conn.commit()
conn.close()

In [14]:
conn = sqlite3.connect('../dbs/calls.db')
query = "SELECT * FROM calls"
df = pd.read_sql_query(query, conn)
df

,id,date,address,expected_launch_time,expected_launch_time_ts,s_mm2,s_ma2,s_fa2,s_q,s_sni,mcap_num,liq_num,launched,launch_time,buy,buy_time,buy_price
0,215601,2024-04-18 14:22:16,FZ2FNQKY84oHx2y3vGe9cq9YymFSTzFFjqBatyN6bDpP,42 seconds ago,2024-04-18 14:21:34,0,0,0,0,3,329,461,None,None,None,None,None
1,215602,2024-04-18 14:22:20,3TDqQCyxdc7pTNQZ2E2Vycdh1X8Wqrc7P3x2wMwoh4pz,In 1 minute,2024-04-18 14:23:20,1,0,0,1,1,8820,17640,None,None,None,None,None
2,215603,2024-04-18 14:22:29,7mjjbNaQBCZGLSNwbfWZH4tCwNw5VZcNqbQGj299qGk4,2 minutes ago,2024-04-18 14:20:29,0,0,0,0,4,1460,0,None,None,None,None,None
3,215604,2024-04-18 14:22:53,HJoaBRWFu44tamHWGX9csxiE9Z322WUMaEwVrUdckALH,6 seconds ago,2024-04-18 14:22:47,1,1,0,0,3,678,1360,None,None,None,None,None
4,215605,2024-04-18 14:23:50,FybqbAftE88wSdr495baGGL1ajRSdP4218PYtm4GH6F4,In 2 minutes,2024-04-18 14:25:50,0,0,0,2,0,2710,5430,None,None,None,None,None


In [15]:

conn.close()